In [ ]:
from importlib import reload
from environment import lmmsabr
from environment.lmmsabr import make_nss_yield_df, compute_6m_forward_dataframe
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=10, suppress=True)
np.set_printoptions(threshold=np.inf, linewidth=np.inf)
reload(lmmsabr)
lmm = lmmsabr.LMMSABR(imm=True,tenor=5, resolution=126, tau=0.5,sim_time=0.25, swap_client_expiry=1, swap_hedge_expiry=2,beta=0.5)

To isolate the effects of vol regimes, pick a single starting curve

In [ ]:
df = compute_6m_forward_dataframe(make_nss_yield_df()).iloc[[0]]
df[:] = 0.05

the correlation will stay constant across regimes

In [ ]:
rho_config = {'eta_range':(0.0,0.0), 'lambda_range':( 0.0242, 0.0242)}
theta_config = {'eta_range':(0.0,0.0), 'lambda_range':( 2*0.0248, 2*0.0248)}
phi_config = {'phi_long':-0.6,'phi_short':-0.5, 'lambda3_range': (0.0,0.0), 'lambda4_range':(0.0,0.0)}

lmm.sample_starting_conditions(df,
                               rho_kwargs=rho_config,
                               theta_kwargs=theta_config,
                               phi_kwargs=phi_config)
lmm.prime()

In [ ]:
rng = np.random.default_rng(42)
#rng.multivariate_normal(np.zeros(2 * lmm.num_forwards), lmm.sigma_big*1/252, (lmm.swap_sim_shape[0] - 1 * lmm.sub_steps))
lmm.plot(lmm.sigma_big)

In [ ]:
def make_plots():
    lmm.simulate(seed=0)
    lmm.get_swap_matrix()
    lmm.get_sabr_params_imm()
    lmm.plot(lmm.h_mat)
    lmm.plot(lmm.g_mat)
    print("f_sim")
    lmm.plot(lmm.f_sim[:,lmm.ids_fwd_canon])
    print("s_mat")
    lmm.plot(lmm.k_mat[:,lmm.ids_fwd_canon]*lmm.g_mat[:,lmm.ids_fwd_canon])
    plt.show()
    plt.plot(lmm.V)
    plt.show()
    plt.plot(lmm.alpha)
    plt.show()
    plt.plot(lmm.phi)
    plt.show()
    n_sim = 100
    swap_paths_hed = np.zeros((n_sim,lmm.swap_sim_shape[0]))
    swap_paths_liab  = np.zeros((n_sim,lmm.swap_sim_shape[0]))
    for i in range(n_sim):
        lmm.simulate()
        lmm.get_swap_matrix()
        swap_paths_hed[i] = lmm.swap_sim[:,0]
        swap_paths_liab[i] = lmm.swap_sim[:,1]
    plt.plot(swap_paths_hed.T)
    plt.show()
    plt.plot(swap_paths_liab.T)
    plt.show()

### Calm regime

Start by plotting the interesting features in the calm regime

In [ ]:


# lmm.generate_episodes_imm(n_episodes=25_000, out_dir='data/calm_flat_5pct_curve')
0

### Stressed regime

In [ ]:
#lmm.generate_episodes_imm(n_episodes=25_000, max_stress_prob=1,min_stress_prob=1, out_dir='data/stress_flat_5pct_curve')
lmm.switch_regime(stressed=True)
make_plots()

### Curriculum calm->stress regime

In [ ]:
#lmm.generate_episodes_imm(n_episodes=20_000, max_stress_prob=0.8,min_stress_prob=0, out_dir='data/curriculum_flat_5pct_curve') # only 20000, eval will be 50/50

### 50/50 test set for evaluation across training methods

In [ ]:
#lmm.generate_episodes_imm(n_episodes=5000, max_stress_prob=0.5,min_stress_prob=0.5, out_dir='data/5050_flat_5pct_curve')